# Install libraries

In [ ]:
!pip install tensorflow
!pip install tensorflow_text
!pip install tensorflow_hub

# Import Libraries

In [28]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [29]:
import pandas as pd

df = pd.read_csv("spam.csv")
df.head(5)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


Columns 2,3,4 contain no important data and can be deleted.
Also, we rename column v1 as “category” and v2 as “message”.

In [30]:
df = df.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
df = df.rename(columns={"v1":"category", "v2":"message"})

After deletion and renaming the column data will look like this

In [31]:
df.head(10)

,category,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [32]:
df.groupby('category').describe()

message                                                            \
           count unique                                                top   
category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    653  Please call our customer service representativ...   

               
         freq  
category       
ham        30  
spam        4

Creating a new column, if the value is spam the value of this spam column will be 1 and for ham, it will be zero.

In [33]:
df['spam']=df['category'].apply(lambda x: 1 if x=='spam' else 0)
df.head()

,category,message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [34]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['message'],df['spam'], stratify=df['spam'])

In [35]:
X_train.head(4)

138     You'll not rcv any more msgs from the chat svc...
511         Prabha..i'm soryda..realy..frm heart i'm sory
3959    I sent lanre fakeye's Eckankar details to the ...
2532                                            Yup ok...
Name: message, dtype: object

# Creating embedding using BERT
Using pre-trained BERT encoder and pre-processor. It will take time to download the pre-trained BERT model.

In [36]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

get_sentence_embeding([
    "500$ discount. hurry up", 
    "Bhavin, are you up for a volleybal game tomorrow?"]
)

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.84351707, -0.5132728 , -0.8884573 , ..., -0.74748874,
        -0.7531474 ,  0.91964495],
       [-0.87208354, -0.50543964, -0.9444667 , ..., -0.858475  ,
        -0.7174534 ,  0.88082975]], dtype=float32)>

# Create Model

In [37]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [39]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           text[0][0]                       
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'sequence_output':  109482241   keras_layer[1][0]                
                                                                 keras_layer[1][1]                
                                                                 keras_layer[1][2]                
____________________________________________________________________________________________

In [40]:
len(X_train)

4179

In [41]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [25]:

model.fit(X_train, y_train, epochs=5)

Epoch 1/5
131/131 [==============================] - 984s 7s/step - loss: 0.3304 - accuracy: 0.8665
Epoch 2/5
131/131 [==============================] - 978s 7s/step - loss: 0.2386 - accuracy: 0.8945
Epoch 3/5
131/131 [==============================] - 979s 7s/step - loss: 0.2018 - accuracy: 0.9172
Epoch 4/5
131/131 [==============================] - 992s 8s/step - loss: 0.1798 - accuracy: 0.9325
Epoch 5/5
131/131 [==============================] - 1027s 8s/step - loss: 0.1622 - accuracy: 0.9435


In [26]:
model.evaluate(X_test, y_test)

44/44 [==============================] - 325s 7s/step - loss: 0.1576 - accuracy: 0.9354


[0.15759383141994476, 0.9353912472724915]

# Prediction

In Sigmoid whenever the value is more than 0.5 that means it is spam and the value which is less than 0.5 means no spam.


In [27]:
reviews = [
    'Reply to win Â£100 weekly! Where will the 2006 FIFA World Cup be held? Send STOP to 87239 to end service',
    'You are awarded a SiPix Digital Camera! call 09061221061 from landline. Delivery within 28days. T Cs Box177. M221BP. 2yr warranty. 150ppm. 16 . p pÂ£3.99',
    'it to 80488. Your 500 free text messages are valid until 31 December 2005.',
    'Hey Sam, Are you coming for a cricket game tomorrow',
    "Why don't you wait 'til at least wednesday to see if you get your ."
]
model.predict(reviews)

array([[0.48746225],
       [0.5439128 ],
       [0.3940646 ],
       [0.03828368],
       [0.01623595]], dtype=float32)